In [4]:
!pip install "domino[clip,text] @ git+https://github.com/HazyResearch/domino@main"


  Cloning https://github.com/HazyResearch/domino (to revision main) to /tmp/pip-install-67uu61rh/domino_bb19155eae6146e29ba21ee3f4f3887c
  Running command git clone --filter=blob:none --quiet https://github.com/HazyResearch/domino /tmp/pip-install-67uu61rh/domino_bb19155eae6146e29ba21ee3f4f3887c
  Resolved https://github.com/HazyResearch/domino to commit 27eb443d1a6b425eedcafa06b1ae2944008b7f27
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 60.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [825 lines of output]
      <string>:17: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      Partial import of sklearn during the bui

In [1]:
!pip install pydantic==1.10.8

In [4]:
!pip install meerkat-ml

In [2]:
# !pip install scikit-learn==0.24.2

In [4]:
# import meerkat
from domino._slice.mixture import DominoSlicer
import numpy as np


ImportError: cannot import name 'ModelField' from 'pydantic.fields' (/root/miniconda3/lib/python3.8/site-packages/pydantic/fields.py)

In [3]:
from domino._slice.mixture import DominoSlicer
import numpy as np

places8_clip_emd = "places8_image_features_clip.npy"
places8_image_preds = "places8_image_preds_1hot.npy"
places8_targets = "places8_image_targets.npy"
clip_emb = np.load(places8_clip_emd)
image_1hot_preds = np.load(places8_image_preds)
places8_targets = np.load(places8_targets)

indexes_1 = np.where(places8_targets==1)
print(indexes_1)
clip_emb_1 = clip_emb[indexes_1]
image_1hot_preds_1 = image_1hot_preds[indexes_1]
places8_targets_1 = places8_targets[indexes_1]
print(f"Embed shape {clip_emb_1.shape} | target shape {places8_targets_1.shape} | predic 1hot encode {image_1hot_preds_1.shape}")
# print(f"Embed shape {clip_emb.shape} | target shape {places8_targets.shape} | predic 1hot encode {image_1hot_preds.shape}")

(array([ 35998,  35999,  36000, ..., 364803, 364804, 364805]),)
Embed shape (100012, 512) | target shape (100012,) | predic 1hot encode (100012, 8)


In [3]:
domino = DominoSlicer(
    y_log_likelihood_weight=40,
    y_hat_log_likelihood_weight=40,
    n_mixture_components=65,
    n_slices=5
)
domino.fit(embeddings = clip_emb, targets = places8_targets, pred_probs = image_1hot_preds)

NameError: name 'DominoSlicer' is not defined